In [1]:
import datetime
import random

import numpy as np
from keras.layers import Activation
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dense
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential, Model
from keras.applications.resnet50 import ResNet50
from keras.applications import VGG16
from keras.applications.inception_v3 import InceptionV3
from tqdm import tqdm

from common import *

Using TensorFlow backend.


In [2]:
print(".. setting base configuration & params ..")

initialize()

best_model_filepath = './saved-models/achitecture-01.hdf5'
training_epochs = 50
batch_size = 20
images_size = 270


.. setting base configuration & params ..


In [3]:
(train_tensors, X_train, y_train,
test_tensors, X_test, y_test, 
valid_tensors, X_validate, y_validate, 
duration_loading) = load_and_split_data(images_size=images_size)

.. loading & splitting data ..


100%|██████████| 352/352 [00:00<00:00, 548.93it/s]


In [4]:
print(".. constructing the model ..")

model = Sequential()

model.add(Conv2D(filters=16, kernel_size=2, padding='same', input_shape=(images_size, images_size, 3), 
                 data_format="channels_last"))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=32, kernel_size=2, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=2, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(GlobalAveragePooling2D())
model.add(Dense(2, activation='softmax'))

model.summary()

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])


.. constructing the model ..
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 270, 270, 16)      208       
_________________________________________________________________
batch_normalization_1 (Batch (None, 270, 270, 16)      64        
_________________________________________________________________
activation_1 (Activation)    (None, 270, 270, 16)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 135, 135, 16)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 135, 135, 32)      2080      
_________________________________________________________________
batch_normalization_2 (Batch (None, 135, 135, 32)      128       
_________________________________________________________________
activation_2 (Activation)    (None, 135, 135, 3

In [5]:
duration_training = train_single_model(model, 
                                       best_model_filepath, 
                                       train_tensors, y_train, 
                                       valid_tensors, y_validate, 
                                       training_epochs, 
                                       batch_size)


.. training the model ..
Train on 1408 samples, validate on 352 samples
Epoch 1/50
1408/1408 [==============================] - 461s 327ms/step - loss: 0.4370 - acc: 0.7834 - val_loss: 0.4045 - val_acc: 0.8494

Epoch 00001: val_loss improved from inf to 0.40453, saving model to ./saved-models/achitecture-01.hdf5
Epoch 2/50
1408/1408 [==============================] - 471s 335ms/step - loss: 0.3458 - acc: 0.8310 - val_loss: 0.4149 - val_acc: 0.8295

Epoch 00002: val_loss did not improve from 0.40453
Epoch 3/50
1408/1408 [==============================] - 469s 333ms/step - loss: 0.3229 - acc: 0.8402 - val_loss: 0.2815 - val_acc: 0.9034

Epoch 00003: val_loss improved from 0.40453 to 0.28148, saving model to ./saved-models/achitecture-01.hdf5
Epoch 4/50
1408/1408 [==============================] - 467s 332ms/step - loss: 0.2949 - acc: 0.8622 - val_loss: 0.2619 - val_acc: 0.8381

Epoch 00004: val_loss improved from 0.28148 to 0.26194, saving model to ./saved-models/achitecture-01.hdf5
Epoc


Epoch 00041: val_loss did not improve from 0.18054
Epoch 42/50
1408/1408 [==============================] - 670s 476ms/step - loss: 0.2102 - acc: 0.9027 - val_loss: 0.3036 - val_acc: 0.8722

Epoch 00042: val_loss did not improve from 0.18054
Epoch 43/50
1408/1408 [==============================] - 681s 484ms/step - loss: 0.2091 - acc: 0.8984 - val_loss: 0.3534 - val_acc: 0.8551

Epoch 00043: val_loss did not improve from 0.18054
Epoch 44/50
1408/1408 [==============================] - 670s 476ms/step - loss: 0.2060 - acc: 0.9077 - val_loss: 0.2060 - val_acc: 0.9091

Epoch 00044: val_loss did not improve from 0.18054
Epoch 45/50
1408/1408 [==============================] - 681s 484ms/step - loss: 0.2012 - acc: 0.9084 - val_loss: 0.2598 - val_acc: 0.8920

Epoch 00045: val_loss did not improve from 0.18054
Epoch 46/50
1408/1408 [==============================] - 658s 468ms/step - loss: 0.2066 - acc: 0.9013 - val_loss: 0.1867 - val_acc: 0.9205

Epoch 00046: val_loss did not improve from 0

In [6]:
print(".. loading best weights ..")

model.load_weights(best_model_filepath)

.. loading best weights ..


In [7]:
print(".. testing the model on Test data ..")
section_start_time = datetime.datetime.utcnow()

test_data_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in tqdm(test_tensors)]

test_data_predictions_time = (datetime.datetime.utcnow() - section_start_time).total_seconds()

  0%|          | 0/440 [00:00<?, ?it/s]

.. testing the model on Test data ..


100%|██████████| 440/440 [01:40<00:00,  4.44it/s]


In [8]:
print(".. testing the model on Training data ..")
section_start_time = datetime.datetime.utcnow()

train_data_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in tqdm(train_tensors)]

train_data_predictions_time = (datetime.datetime.utcnow() - section_start_time).total_seconds()

  0%|          | 0/1408 [00:00<?, ?it/s]

.. testing the model on Training data ..


100%|██████████| 1408/1408 [05:24<00:00,  4.56it/s]


In [9]:
print(".. testing the model on Validation data ..")
section_start_time = datetime.datetime.utcnow()

validation_data_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in tqdm(valid_tensors)]

validation_data_predictions_time = (datetime.datetime.utcnow() - section_start_time).total_seconds()

  0%|          | 0/352 [00:00<?, ?it/s]

.. testing the model on Validation data ..


100%|██████████| 352/352 [01:19<00:00,  4.54it/s]


In [10]:
print("________________________________________________________")
print("TIMING SUMMARY:\n")
print("loading duration: {0:.1f} seconds".format(duration_loading))
print("training duration: {0:.1f} seconds".format(duration_training))

print("________________________________________________________")
print("MODEL PERFORMANCE ON TEST DATA:\n")
print("predict duration: {0:.1f} seconds".format(test_data_predictions_time))
false_positive_images, false_negative_images = summarize_model_performance(X_test, y_test, test_data_predictions)

print("________________________________________________________")
print("MODEL PERFORMANCE ON TRAIN DATA:\n")
print("predict duration: {0:.1f} seconds".format(train_data_predictions_time))
_, _ = summarize_model_performance(X_train, y_train, train_data_predictions)

print("________________________________________________________")
print("MODEL PERFORMANCE ON VALIDATION DATA:\n")
print("predict duration: {0:.1f} seconds".format(validation_data_predictions_time))
_, _ = summarize_model_performance(X_validate, y_validate, validation_data_predictions)

________________________________________________________
TIMING SUMMARY:

loading duration: 6.7 seconds
training duration: 31363.9 seconds
________________________________________________________
MODEL PERFORMANCE ON TEST DATA:

predict duration: 100.6 seconds
all:  440
all_positives:  81
all_negatives:  359
true_positives:  70
true_negatives:  341
false_positives:  18
false_negatives:  11
RECALL: 0.86
SPECIFICITY: 0.95
ACCURACY: 0.93
PRECISION: 0.80
F1 SCORE: 0.83
FP RATE / ERROR I: 0.05
FN RATE / ERROR II: 0.14
________________________________________________________
MODEL PERFORMANCE ON TRAIN DATA:

predict duration: 324.3 seconds
all:  1408
all_positives:  316
all_negatives:  1092
true_positives:  255
true_negatives:  1026
false_positives:  66
false_negatives:  61
RECALL: 0.81
SPECIFICITY: 0.94
ACCURACY: 0.91
PRECISION: 0.79
F1 SCORE: 0.80
FP RATE / ERROR I: 0.06
FN RATE / ERROR II: 0.19
________________________________________________________
MODEL PERFORMANCE ON VALIDATION DATA:


## False Positives (first 20):

In [11]:
display_images(false_positive_images[:20])

## False Negatives (first 20):

In [12]:
display_images(false_negative_images[:20])